# Anagrams with Test-Driven Development

I'm working through a coding exercise from [Exercism](https://exercism.io/) in Clojure and this is the current challenge. 

I thought it would be interesting to work through it in JavaScript and add test cases ones by one so that we refactor the code as we go (instead of being presented with all of the tests at the same time which would encourage us to implement them all at once). This way we can see the code evolve in small steps.

Test-Driven Development is about taking small steps, with tests to guide you as you go.

Here is the challenge:

>Introduction
>
>Given a word and a list of possible anagrams, select the correct sublist.
>
>Given "listen" and a list of candidates like "enlists" "google" "inlets" "banana" the program should return a list containing "inlets".

If we were doing this with a command line project and npm I would use something like _jest_. Since that's usually run from the command line, and this is a notebook, I'll query the result with a helper function.

We start with a failing test ...

In [1]:
function anagramFor(word, candidates) {
    return [];
}

In [2]:
function arrayEquals(a, b) {
    if (a.length != b.length) {
        return false;
    }
    for(let i=0; i < a.length; i++) {
        if (a[i] != b[i]) {
            return false;
        }
    }
    return true;
}

In [3]:
// Test case from the introduction ...
arrayEquals(["inlets"], anagramFor("listen", ["enlists", "google", "inlets", "banana"]))

false

In [4]:
// Simplest thing ...
function anagramFor(word, candidates) {
    return ["inlets"];
}

In [5]:
arrayEquals(["inlets"], anagramFor("listen", ["enlists", "google", "inlets", "banana"]))

true

In [6]:
function testsPass(scenarios) {
    let failed = false;
    for([origin, candidates, anagrams] of scenarios) {
        const actual = anagramFor(origin, candidates);
        if (!arrayEquals(anagrams, actual)) {
            console.log(`Failed: origin=${origin}, candidates=${candidates}; actual=${actual}`);
            failed = true;
        } else {
            console.log(`Passed: origin=${origin}, candidates=${candidates}`);
        }
    }
    return !failed;
}


In [7]:
// Use `var` for scenarios because Node won't allow us to redeclare if we use let (or const)
var scenarios = [
    // origin, candidates, anagrams
    ["listen", ["enlists", "google", "inlets", "banana", "tinsel"], ["inlets", "tinsel"]],
    ["listen", ["enlists", "google", "inlets", "banana"], ["inlets"]],
];
testsPass(scenarios)

Failed: origin=listen, candidates=enlists,google,inlets,banana,tinsel; actual=inlets
Passed: origin=listen, candidates=enlists,google,inlets,banana


false

In [8]:
// Fix the test 
function anagramFor(word, candidates) {
    const sortLettersAlphabetically = (w) => w.split('').sort().join('');
    const sortedWord = sortLettersAlphabetically(word);
    return candidates.filter((w) => sortedWord === sortLettersAlphabetically(w));
}

In [9]:
anagramFor("listen", ["enlists", "google", "inlets", "banana", "tinsel"])

[ 'inlets', 'tinsel' ]

In [10]:
var scenarios = [
    // origin, candidates, anagrams
    ["listen", ["enlists", "google", "inlets", "banana", "tinsel"], ["inlets", "tinsel"]],
    ["listen", ["enlists", "google", "inlets", "banana"], ["inlets"]],
];
testsPass(scenarios)

Passed: origin=listen, candidates=enlists,google,inlets,banana,tinsel
Passed: origin=listen, candidates=enlists,google,inlets,banana


true

Check additional scenarios ...

In [11]:
var scenarios = [
    // origin, candidates, anagrams
    ["diaper", ["hello", "world", "zombies", "pants"],              []],
    ["ant",    ["tan", "stand", "at"],                              ["tan"]]
];
testsPass(scenarios)

Passed: origin=diaper, candidates=hello,world,zombies,pants
Passed: origin=ant, candidates=tan,stand,at


true

In [12]:
var scenarios = [
    ["good", ["dog", "goody"], []]
];
testsPass(scenarios)

Passed: origin=good, candidates=dog,goody


true

In [13]:
var scenarios = [
    ["Orchestra", ["cashregister", "Carthorse", "radishes"],        ["Carthorse"]]
];
testsPass(scenarios)

Failed: origin=Orchestra, candidates=cashregister,Carthorse,radishes; actual=


false

Looks like we need to do a case-insensitive match and than convert to initial caps if the original was in initial caps

In [14]:
function anagramFor(word, candidates) {
    const sortLettersAlphabetically = (w) => w.toLowerCase().split('').sort().join('');
    const capitalize = (w) => w.charAt(0).toUpperCase() + w.substring(1);
    const sortedWord = sortLettersAlphabetically(word);
    const results = candidates.filter((w) => sortedWord === sortLettersAlphabetically(w));
    return (word.charAt(0).toUpperCase() == word.charAt(0)) 
        ? results.map((w) => capitalize(w)) 
        : results;
}

In [15]:
var scenarios = [
    ["Orchestra", ["cashregister", "Carthorse", "radishes"],        ["Carthorse"]]
];
testsPass(scenarios)

Passed: origin=Orchestra, candidates=cashregister,Carthorse,radishes


true

In [16]:
// Initially I put the toLowerCase at the end; why doesn't this work?
var brokenSorter = (w) => w.split('').sort().join('').toLowerCase();

In [17]:
// brokenSorter('Hello')

In [18]:
// Double-check the capitalize function
var capitalize = (w) => w.charAt(0).toUpperCase() + w.substring(1);

Run a [regression test](https://en.wikipedia.org/wiki/Regression_testing) ...

In [19]:
var scenarios = [
    // origin, candidates, anagrams
    ["listen", ["enlists", "google", "inlets", "banana", "tinsel"], ["inlets", "tinsel"]],
    ["listen", ["enlists", "google", "inlets", "banana"], ["inlets"]],
    ["Orchestra", ["cashregister", "Carthorse", "radishes"], ["Carthorse"]],
    ["good", ["dog", "goody"], []],
    ["diaper", ["hello", "world", "zombies", "pants"], []],
    ["ant",    ["tan", "stand", "at"], ["tan"]],
    
];
testsPass(scenarios)

Passed: origin=listen, candidates=enlists,google,inlets,banana,tinsel
Passed: origin=listen, candidates=enlists,google,inlets,banana
Passed: origin=Orchestra, candidates=cashregister,Carthorse,radishes
Passed: origin=good, candidates=dog,goody
Passed: origin=diaper, candidates=hello,world,zombies,pants
Passed: origin=ant, candidates=tan,stand,at


true

In [20]:
// A word cannot be its own anagram
var scenarios = [
    ["banana", ["banana"], []],
];
testsPass(scenarios)

Failed: origin=banana, candidates=banana; actual=banana


false

In [21]:
function anagramFor(word, candidates) {
    const sortLettersAlphabetically = (w) => w.toLowerCase().split('').sort().join('');
    const capitalize = (w) => w.charAt(0).toUpperCase() + w.substring(1);
    const sortedWord = sortLettersAlphabetically(word);
    let results = candidates.filter((w) => sortedWord === sortLettersAlphabetically(w));
    results = results.filter((w) => w != word);
    return (word.charAt(0).toUpperCase() == word.charAt(0)) 
        ? results.map((w) => capitalize(w)) 
        : results;
}

In [22]:
var scenarios = [
    ["banana", ["banana"], []],
];
testsPass(scenarios)

Passed: origin=banana, candidates=banana


true

In [23]:
// Mixed case?
var scenarios = [
    ["BANANA", ["banana"], []],
];
testsPass(scenarios)

Failed: origin=BANANA, candidates=banana; actual=Banana


false

In [24]:
/* Can you fix this one?
function anagramFor(word, candidates) {
    const sortLettersAlphabetically = (w) => w.toLowerCase().split('').sort().join('');
    const capitalize = (w) => w.charAt(0).toUpperCase() + w.substring(1);
    const sortedWord = sortLettersAlphabetically(word);
    let results = candidates.filter((w) => sortedWord === sortLettersAlphabetically(w));
    results = results.filter((w) => w != word);
    return (word.charAt(0).toUpperCase() == word.charAt(0)) 
        ? results.map((w) => capitalize(w)) 
        : results;
}
*/

In [26]:
var scenarios = [
    // origin, candidates, anagrams
    ["listen", ["enlists", "google", "inlets", "banana", "tinsel"], ["inlets", "tinsel"]],
    ["listen", ["enlists", "google", "inlets", "banana"], ["inlets"]],
    ["Orchestra", ["cashregister", "Carthorse", "radishes"], ["Carthorse"]],
    ["good", ["dog", "goody"], []],
    ["diaper", ["hello", "world", "zombies", "pants"], []],
    ["ant",    ["tan", "stand", "at"], ["tan"]],
    ["banana", ["banana"], []],
    ["BANANA", ["banana"], []],
];
testsPass(scenarios)

Passed: origin=listen, candidates=enlists,google,inlets,banana,tinsel
Passed: origin=listen, candidates=enlists,google,inlets,banana
Passed: origin=Orchestra, candidates=cashregister,Carthorse,radishes
Passed: origin=good, candidates=dog,goody
Passed: origin=diaper, candidates=hello,world,zombies,pants
Passed: origin=ant, candidates=tan,stand,at
Passed: origin=banana, candidates=banana
Failed: origin=BANANA, candidates=banana; actual=Banana


false

We have quite a few test scenarios now. Are any of them redundant? Particularly if these were integration tests that are time-consuming to run, can we remove any unnecessary tests?

Can you think of any more scenarios to test? How should we handle an origin of _"isn't"_ and candidates of _["tins", "nits", "sin"]_How would you handle invalid inputs such as numbers, or _undefined_ ? You can add more cells below and experiment.